# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 4

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task4`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [ ]:
!az login

## Step 1. Write the code

In [ ]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

In [ ]:
teamname="msft_is_the_best"  # Update this field with your team name
task=["task4"]
slack_id="myslackid"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [ ]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task4_DumpMachineWrapper : qsharp.QSharpCallable = None
Task4_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 4 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [ ]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;

// Task 4. 
// (input will contain 7 qubits)
operation Task4(input : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(input);
    for i in 0 .. 3 .. 2^N - 1 {
        ControlledOnInt(i, X)(input, target);
    }
}

In [ ]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task4_DumpMachineWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task4(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task4_ResourceEstimationWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    Task4(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [ ]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
qsharp.config["dump.truncateSmallAmplitudes"]=True
Task4_DumpMachineWrapper.simulate()

## Step 3. Evaluate the code using resource estimation

In [ ]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="...",
    location="...",
    credential="CLI")

In [ ]:
qsharp.azure.target("microsoft.estimator")

In [ ]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task4_ResourceEstimationWrapper, jobName="RE for the task 4")

In [ ]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

In [ ]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [ ]:
evaluate_results(result)